In [ ]:
%load configuration.py


In [ ]:
%load data.py


In [ ]:
%load setup-pd.py


In [ ]:
import datetime, pytz

df_ip['utcoffset'] = df_ip.timezone.apply(
    lambda x : datetime.datetime.now(pytz.timezone(x)).strftime('%z') if isinstance(x, str) else None
)

df_ip = df_ip[['ip', 'city', 'region', 'longitude', 'latitude', 'org', 'postal', 'timezone', 'utcoffset', 'country']]
#df_ip.to_csv('/home/max/jupyter/dataplane/raw/ip_loc.csv')


In [ ]:
df = df_log[df_log.category == 'ssh'][['origin', 'origin_id', 'timestamp', 'ip']]
#df.groupby()

#df['utcoffset'] = df_

def utc2number(x):
    sign = -1 if x[0] == '-' else 1
    hour = int(x[1:3])
    minute = int(x[3:])
    minute /= 60
    return sign * (hour + minute)
    

df = df.merge(df_ip, how='left', left_on='ip', right_on='ip')
df = df[['origin', 'origin_id', 'timestamp', 'ip', 'utcoffset', 'country']]
#df['utcoffset'] = df.utcoffset.apply(utc2number)
#df['utcoffset'] = df.utcoffset.apply(lambda x : f"{x[0:3]}:{x[3:]}")
df['utcoffset'] = df.utcoffset.apply(lambda x : f"{x[0:3]}:00")
df = df.set_index(df.timestamp)
df = df.drop(['timestamp'], axis=1)
df = df.groupby([pd.Grouper(freq='H'), 'origin', 'origin_id', 'ip', 'utcoffset']).count()
df = df.rename(columns={'country' : 'ips'})
df = df.reset_index().set_index('timestamp')
df = df.groupby([df.index.hour, 'origin', 'origin_id', 'utcoffset']).sum()
df = df.reset_index().set_index('timestamp')
df


In [ ]:
#plots = {}

def order_utcoffsets(index):
    ordered = [x for x in index if x[0] == '-']
    ordered.reverse()
    ordered += [x for x in index if x[0] == '+']
    ordered = pd.Index(ordered)
    return ordered

tmp = df.groupby([df.index, 'utcoffset']).sum()
tmp = tmp[['ips']].unstack('utcoffset')
tmp = tmp.fillna(0).astype(int)
tmp = pd.DataFrame(tmp.to_dict()).transpose()
tmp = tmp.reset_index().drop('level_0', axis=1).set_index('level_1')
tmp = pd.DataFrame(tmp.to_dict())
tmp = tmp.reindex(order_utcoffsets(tmp.index))
tmp.style.background_gradient(cmap='Wistia', vmax=3000)

In [ ]:
timezones = df_ip[['timezone', 'utcoffset']].dropna().drop_duplicates().set_index('utcoffset').sort_index()
timezones
